In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain.chat_models import init_chat_model

llm_lite = init_chat_model("google_genai:gemini-2.5-flash-lite", temperature=0.8)
response = llm_lite.invoke("Hey how are you?")
response.text

"I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you.\n\nHow about you? How are you doing today? Is there anything I can help you with?"

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0.8)
response = llm.invoke("Hey how are you?")
response.text

"Hey there! I'm doing great, thanks for asking!\n\nHow about you? How are you doing today?"

In [ ]:
from langchain.chat_models import init_chat_model

llm_2 = init_chat_model("claude-haiku-4-5-20251001", temperature=0)
response = llm_2.invoke("Hey how are you?")
response.text

"I'm doing well, thank you for asking! I'm here and ready to help with whatever you need - whether that's having a conversation, answering questions, or working on a task together. How are you doing today?"

In [5]:
response = llm_2.invoke("How is the wheater un New York City?")
response.text

'I don\'t have access to real-time weather data, so I can\'t tell you the current weather in New York City. \n\nFor the most up-to-date weather information, I\'d recommend:\n- Checking weather.com or weather.gov\n- Using a weather app on your phone\n- Searching "NYC weather" in your web browser\n- Asking a voice assistant with current data access\n\nIs there anything else about New York City I can help you with, such as typical weather patterns for different seasons or climate information?'

In [6]:
system_prompt = """
You are a sales assistant who helps customers find the products they need.

And your products are:
- Computer
- Mouse
- Keyboard
- Headphones
- Mousepad
- Monitor
"""
messages = [
    ("system", system_prompt),
    ("user", "Tell me the products you have in the store")
]
response = llm_2.invoke(messages)
response.text

"I can help you with that! Here are the products we have available in our store:\n\n1. **Computer** - Desktop or laptop systems for your computing needs\n2. **Mouse** - For precise cursor control and navigation\n3. **Keyboard** - For typing and input commands\n4. **Headphones** - For audio listening and communication\n5. **Mousepad** - To provide a smooth surface for your mouse\n6. **Monitor** - Display screens for your computer setup\n\nIs there any specific product you're interested in learning more about, or are you looking for something particular for your setup?"

In [7]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    products = [
        {"name": "Computer", "price": 1500},
        {"name": "Mouse", "price": 150},
        {"name": "Keyboard", "price": 500},
        {"name": "Mousepad", "price": 8400},
        {"name": "Monitor", "price": 10}
    ]
    return "".join([f"{product['name']} - {product['price']}" for product in products])



In [8]:
get_products.invoke({})

'Computer - 1500Mouse - 150Keyboard - 500Mousepad - 8400Monitor - 10'

In [9]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    response = requests.get("https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])



In [10]:
get_products.invoke({})

'Classic Red Baseball Cap - 123123123Classic Black Baseball Cap - 58Classic Olive Chino Shorts - 84Classic White Crew Neck T-Shirt - 39Classic White Tee - Timeless Style and Comfort - 73Classic Black T-Shirt - 35Sleek Wireless Headphone & Inked Earbud Set - 44Sleek Comfort-Fit Over-Ear Headphones - 28Efficient 2-Slice Toaster - 48Sleek Wireless Computer Mouse - 10Sleek Modern Laptop with Ambient Lighting - 43Sleek Modern Laptop for Professionals - 97Stylish Red & Silver Over-Ear Headphones - 39Sleek Mirror Finish Phone Case - 27Sleek Smartwatch with Vibrant Display - 16Sleek Modern Leather Sofa - 53Mid-Century Modern Wooden Dining Table - 24Elegant Golden-Base Stone Top Dining Table - 66Modern Elegance Teal Armchair - 25Elegant Solid Wood Dining Table - 67Modern Minimalist Workstation Setup - 49Modern Ergonomic Office Chair - 71Futuristic Holographic Soccer Cleats - 39Rainbow Glitter High Heels - 39Chic Summer Denim Espadrille Sandals - 33Vibrant Runners: Bold Orange & Blue Sneakers - 

In [11]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data["current_weather"]["temperature"]}C with {data["current_weather"]["windspeed"]}km/h of wind."
    return response

get_weather.invoke({"city": "New York"})

'The weather in New York is 11.0C with 11.4km/h of wind.'

In [16]:
system_prompt = """
You are a sales assistant who helps customers find the products they need and provides the city weather.

Your tools are:
- get_products: to get the products you offer in the store
- get_weather: to get the city's weather
"""
messages = [
    ("system", system_prompt),
    ("user", "What products do you offer in your store?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls
# response.text

[{'name': 'get_products',
  'args': {},
  'id': '231665d2-f787-4add-b3fc-207392967fa2',
  'type': 'tool_call'}]

In [17]:
messages = [
    ("system", system_prompt),
    ("user", "Hey how are you?")
]
response = llm_with_tools.invoke(messages)
response.text

"I'm doing great! How can I help you today? Are you looking for any particular products, or are you interested in checking the weather?"

In [14]:
system_prompt = """
You are a sales assistant who helps customers find the products they need and provides the city weather.

Your tools are:
- get_products: to get the products you offer in the store
- get_weather: to get the city's weather
"""
messages = [
    ("system", system_prompt),
    ("user", "What is the weather in NY city?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_weather',
  'args': {'city': 'NY city'},
  'id': '1f0e98ba-e633-44f8-8745-534048c7c198',
  'type': 'tool_call'}]

In [18]:
system_prompt = """
You are a sales assistant who helps customers find the products they need and provides the city weather.

Your tools are:
- get_products: to get the products you offer in the store
- get_weather: to get the city's weather
"""
messages = [
    ("system", system_prompt),
    ("user", "What is the weather in the capital of Italy?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_weather',
  'args': {'city': 'Rome'},
  'id': 'e99892ea-b023-4c08-b32e-2390c111767e',
  'type': 'tool_call'}]